In [1]:
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .appName("halltape_pyspark")\
        .master("spark://spark-master:7077")\
        .config("spark.driver.memory", "1g")\
        .config("spark.driver.cores", "1")\
        .config("spark.executor.instances", "1")\
        .config("spark.executor.memory", "512m")\
        .config("spark.executor.cores", "2")\
        .getOrCreate()

print(f"Spark UI: http://localhost:4040/jobs/")

print(f"Alternative method: spark.sparkContext.uiWebUrl")

24/11/23 20:13:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/23 20:13:27 WARN JettyUtils: GET /stages/ failed: java.util.NoSuchElementException
java.util.NoSuchElementException
	at org.apache.spark.util.kvstore.InMemoryStore.read(InMemoryStore.java:85)
	at org.apache.spark.status.ElementTrackingStore.read(ElementTrackingStore.scala:121)
	at org.apache.spark.status.AppStatusStore.appSummary(AppStatusStore.scala:522)
	at org.apache.spark.ui.jobs.AllStagesPage.render(AllStagesPage.scala:47)
	at org.apache.spark.ui.WebUI.$anonfun$attachPage$1(WebUI.scala:89)
	at org.apache.spark.ui.JettyUtils$$anon$1.doGet(JettyUtils.scala:80)
	at javax.servlet.http.HttpServlet.service(HttpServlet.java:687)
	at ja

Spark UI: http://localhost:4040/jobs/
Alternative method: spark.sparkContext.uiWebUrl


In [ ]:
# Download this big file 1.38Gb
# https://drive.google.com/file/d/1gzMJ2zgvD6tXOieyOJbsD8I1gI8b0AeU/view?usp=share_link

In [7]:
# spark.sql(''' CLEAR CACHE ''')

spark.catalog.clearCache()

In [7]:
# OUT OF MEMORY

# example good
d = spark.read.csv('data/customs_data.csv', header=True, sep='\t').limit(1)
d.collect()

# df = d.union(d).union(d).union(d).withColumn('test', F.lit('new_column'))

# df.collect()

[Row(direction='ИМ', month='01/2016', country='IT', code='6204695000', measure='ШТ', value='131', netto='1', quantity='7', region='46000', district='01')]

In [4]:
# SPILL SIZE

from pyspark.sql import functions as F

d = spark.read.csv('data/customs_data.csv', header=True, sep='\t')

df = d.union(d).union(d).union(d).withColumn('test', F.lit('new_column'))


# result = df.groupBy("country").agg(F.count("*").alias("total_countries"))


# Выполняем join с использованием псевдонимов
result = df.alias("df1")\
                .join(df.alias("df2"),F.col("df1.country") == F.col("df2.country"), "inner")\
                .select('df1.*')

result.coalesce(1).write.csv('data/test5.csv')

24/11/23 20:17:33 ERROR FileFormatWriter: Aborting job 3318583b-5028-425b-8598-15ce6be0f27d.
org.apache.spark.SparkException: Job 3 cancelled because killed via the Web UI
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2023)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1919)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleStageCancellation$1(DAGScheduler.scala:1908)
	at scala.runtime.java8.JFunction1$mcVI$sp.apply(JFunction1$mcVI$sp.java:23)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofInt.foreach(ArrayOps.scala:246)
	at org.apache.spark.scheduler.DAGScheduler.handleStageCancellation(DAGScheduler.scala:1901)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:2166)
	at org.apache.spark.scheduler.DAGSche

Py4JJavaError: An error occurred while calling o109.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:226)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:178)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:175)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:213)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:210)
	at org.apache.s

24/11/23 20:17:34 WARN TaskSetManager: Lost task 0.0 in stage 5.0 (TID 91, 172.18.0.4, executor 1): TaskKilled (Stage cancelled)
